<h1>Segmenting and Clustring Neighborhood<h1\>

__Importing the necessary libraries__

In [1]:
from bs4 import BeautifulSoup
import requests   
import lxml       
import pandas as pd
import numpy as np
import time
import geocoder
import ssl
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

ssl._create_default_https_context = ssl._create_unverified_context

__Using Beautiful Soup for scraping the website__

In [2]:
website = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website, 'lxml')
table = soup.find('table', class_='wikitable')

__Managening our table to get the information__

In [3]:
tablex = ""
for tr in table.find_all('tr'):
    row = ""
    for tds in tr.find_all('td'):
        row = row + "," + tds.text
    tablex = tablex + row[1:]
print(tablex)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Queen's Park,Not assigned
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James 

__Converting data to .csv file__

In [4]:
csv_data = open('toronto.csv', 'wb')
csv_data.write(bytes(tablex,encoding="ascii",errors="ignore"))

8708

__Converting data to DataFrame__

In [5]:
df = pd.read_csv('toronto.csv', header = None)
df.columns = ['Postalcode', 'Borough', 'Neighbourhood']
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


__Deleting "Not assigned" Borough rows__

In [6]:
indexnum = df[df['Borough'] == 'Not assigned'].index
df.drop(indexnum, inplace = True)
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


__If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.__

In [7]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


__Combinning the same area code rows__

In [8]:
df_combin = df.groupby(['Postalcode', 'Borough'], sort = False).agg( ','.join)
df_2 = df_combin.reset_index()
df_2.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


__Finding out the shape of the data frame by using .shape method__

In [9]:
df_2.shape

(103, 3)

__The Beginning of 2nd Question's Answer__

In [10]:
url = 'http://cocl.us/Geospatial_data'
lat_long_df = pd.read_csv(url)


detailed_df = pd.DataFrame({'Postalcode':df_2['Postalcode'], 
                            'Borough':df_2['Borough'], 
                            'Neighbourhood':df_2['Neighbourhood'], 
                            'Latitude':lat_long_df['Latitude'], 
                            'Longitude':lat_long_df['Longitude']})


print("Toronto has  {} Boroughs and {} Neighborhoods.".format(len(detailed_df.Borough.unique()), len(detailed_df.Neighbourhood.unique())))

Toronto has  11 Boroughs and 102 Neighborhoods.


__Cluster Analysing__

In [11]:
borough = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
for tor in borough:
    print("{} has a total of {} postal codes.".format(tor, detailed_df[detailed_df['Borough'] == tor].Postalcode.count()))

Downtown Toronto has a total of 19 postal codes.
Central Toronto has a total of 9 postal codes.
West Toronto has a total of 6 postal codes.
East Toronto has a total of 5 postal codes.


In [12]:
Downtown_Toronto = df_2[df_2['Borough'] == 'Downtown Toronto']
Central_Toronto= df_2[df_2['Borough'] == 'Central Toronto']
West_Toronto = df_2[df_2['Borough'] == 'West Toronto']
East_Toronto = df_2[df_2['Borough'] == 'East Toronto']

combined = pd.concat([Downtown_Toronto, Central_Toronto, West_Toronto, East_Toronto], sort=False)
toronto_dataframe = combined.reset_index(drop=True)

print(toronto_dataframe.shape)
toronto_dataframe.head(10)

(39, 3)


,Postalcode,Borough,Neighbourhood
0,M5A,Downtown Toronto,Harbourfront
1,M7A,Downtown Toronto,Queen's Park
2,M5B,Downtown Toronto,"Ryerson,Garden District"
3,M5C,Downtown Toronto,St. James Town
4,M5E,Downtown Toronto,Berczy Park
5,M5G,Downtown Toronto,Central Bay Street
6,M6G,Downtown Toronto,Christie
7,M5H,Downtown Toronto,"Adelaide,King,Richmond"
8,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station"
9,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre"


In [13]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}. '.format(latitude, longitude))

Coordinates of Toronto are 43.653963, -79.387207. 


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers
for lat, lng, borough, neighborhood in zip(lat_long_df['Latitude'], lat_long_df['Longitude'], toronto_dataframe['Borough'], toronto_dataframe['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
    
map_toronto

__Using Foursqure for exploring neighborhood__

In [15]:
client_id = 'IESMHKZNKMWB0YY1SYEMABXTVH1EZIJWHCS1VPI115CL2NUB'
client_secret = 'MFZLEQSUITYU3IUAG5JRLAQTEHRLCKYE5TQDOX5PSFJMTX1D'
version = '20180605'

In [16]:
# first neighborhood 
toronto_dataframe.loc[0, 'Neighbourhood']

'Harbourfront'

In [17]:
Harbourfront_latitude = detailed_df.loc[0, 'Latitude']
Harbourfront_longitude = detailed_df.loc[0, 'Longitude']
print('Harbourfront\'s latitude and longitude values are {}, {}.'.format(Harbourfront_latitude, Harbourfront_longitude))

Harbourfront's latitude and longitude values are 43.806686299999996, -79.19435340000001.


__Getting top 100 venues that are in Harbourfront within a radius of 500 meteres__

In [18]:
venues = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secret, version, Harbourfront_latitude, Harbourfront_longitude, radius, venues)

response = requests.get(url).json()
response

{'meta': {'code': 200, 'requestId': '5e442c8add0f850029b7b83a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'